> **Atividade** (Sendo aprimorada):
> 1. Adapte o código acima para fazer com que a função `solve_triangular_superior()` também retorno o número de operações (somas, produtos, divisões e subtrações numéricas) que foram realizadas. (**Dica**: A resposta é $n^2$; tente deduzir este valor formalmente).

In [56]:
import numpy as np

In [57]:
def solve_triangular_superior(U, b):
    '''Resolve um sistema triangular superior do tipo Ux = b.

    Parametros obrigatorios
    ----------
    U : Array-like de dimensao 2
        Matriz quadrada triangular superior inversível

    b : Array-like de dimensão 1
        Vetor independente

    Saída
    ----------
    x : Array-like de dimensão 1
        Solução do sistema Ux = b
    
    num_op : int 
        Número de operações'''

    num_op = 0
    n = U.shape[0]          # Ordem das matrizes
    
    # Cópias usuais para evitar problemas
    x = b.copy().reshape(n)

    # Vai linha-a-linha, de baixo para cima, escalonando a matriz utilizando o pivô na diagonal
                        
    for i in range(n-1,-1,-1):
        x[i] /= U[i,i]     # Normaliza a i-ésima linha

        num_op +=1
        for j in range(i-1,-1,-1):
            x[j] -= U[j,i]*x[i]     # Pivoteia a i-ésima coluna, utilizando a entrada diagonal como pivô
            num_op += 2

    return x, num_op

In [58]:
tabela = []
ordem = 5
for tam in range(2, ordem+1):
    U = 20*np.random.rand(tam,tam) -10 # Matriz triangular aleatória com entradas em [-10,10]

    for i in range(tam):
        for j in range(i):
            U[i,j]=0.0        # Aniquila as entradas abaixo da diagonal principal para deixar triangular superior    
            
    b = 20*np.random.rand(tam) - 10 # Vetor aleatório com entradas em [-10,10]

    x, num_operacoes = solve_triangular_superior(U,b) # Solução pretendida do sistema Ux = b
    tabela.append([tam, num_operacoes])
    # print(f"\nA matriz U é dada por\n{U}")
    # print(f"\nO vetor b é dada por\n{b}")
    # print(f"\nO vetor x é dada por\n{x}")
    # print(f"\nO vetor Ux é dada por\n{U@x}")
    # print(f"---------------------------------")

# print(f"\nTabela relacionando a Ordem da matriz e o número de operações")
# print(f"\n|{'Ordem':^10}|{'Num. Op.':^10}|")

# for linha in tabela:    
#     print(f"|{linha[0]:^10}|{linha[1]:^10}|")

> 2. Adapte o código acima para fazer o processo análogo (também contando operações), mas com matrizes triangulares _inferiores_.

In [59]:
def solve_triangular_inferior(L, b):
    '''Resolve um sistema triangular inferior do tipo Lx = b.

    Parametros obrigatorios
    ----------
    L : Array-like de dimensao 2
        Matriz quadrada triangular inferior inversível

    b : Array-like de dimensão 1
        Vetor independente

    Saída
    ----------
    x : Array-like de dimensão 1
        Solução do sistema Lx = b
    
    num_op : int 
        Número de operações'''

    num_op = 0
    n = L.shape[0]          # Ordem das matrizes
    
    # Cópias usuais para evitar problemas
    x = b.copy().reshape(n)
                        
    for i in range(n):
        for j in range(i):
            x[i] -= L[i][j] * x[j]
            num_op += 2
        x[i] /= L[i][i]
        num_op += 1
    return x, num_op

In [60]:
ordem = 3
L = 20*np.random.rand(ordem,ordem) -10 # Matriz triangular aleatória com entradas em [-10,10]
# print (U)
for i in range(ordem):
    for j in range(ordem-1, i, -1):
        L[i,j]=0.0        # Aniquila as entradas abaixo da diagonal principal para deixar triangular superior

print(L)


# print(U)    
        
b = 20*np.random.rand(ordem) - 10 # Vetor aleatório com entradas em [-10,10]

x, num_operacoes = solve_triangular_inferior(L,b) # Solução pretendida do sistema Ux = b

print(f"\nA matriz L é dada por\n{L}")
print(f"\nO vetor b é dada por\n{b}")
print(f"\nO vetor x é dada por\n{x}")
print(f"\nO vetor Lx é dada por\n{L@x}")

[[ 1.09524703  0.          0.        ]
 [-8.43083515  8.03618762  0.        ]
 [ 0.78191359 -6.82573512 -5.71113891]]

A matriz L é dada por
[[ 1.09524703  0.          0.        ]
 [-8.43083515  8.03618762  0.        ]
 [ 0.78191359 -6.82573512 -5.71113891]]

O vetor b é dada por
[-1.97159744 -5.50358858  8.53536785]

O vetor x é dada por
[-1.8001395  -2.57339287  1.33465091]

O vetor Lx é dada por
[-1.97159744 -5.50358858  8.53536785]


> 3. Adapte o algoritmo de Crout&ndash;Dolittle, para fazer com que ele também retorne o número de operações realizadas.

In [61]:
def crout_dolittle(A):
    ''' Decomposição LU de A pelo algoritmo de Crout'''

    num_op = 0

    m,n = np.shape(A)

    L = np.zeros((n,n))
    U = np.zeros((n,n))

    L[0,0] = 1          # Escolha de Dolittle
    U[0,0] = A[0,0]

    for j in range(1,n):
        U[0,j] = A[0,j]/L[0,0]      # Determina a primeira linha de U
        L[j,0] = A[j,0]/U[0,0]      # Determina a primeira coluna de L
        num_op += 2
    
    for i in range(1,n):
        L[i,i]=1        # Escolha de Dolittle
        U[i,i] = A[i,i] - sum([L[i,k]*U[k,i] for k in range(i)])
        num_op += i + 1

        for j in range(i+1,n):
            U[i,j] = (A[i,j] - sum([L[i,k]*U[k,j] for k in range(i)]))/L[i,i]       # Determina a i-ésima linha de U
            L[j,i] = (A[j][i] - sum([L[j,k]*U[k,i] for k in range(i)]))/U[i,i]      # Determina a i-ésima coluna de L
            num_op += 2*(i + 2)
    return L , U, num_op

In [62]:
tabela = []
ordem = 10
for tam in range(2, ordem+1):
    A = 20*np.random.rand(tam, tam) - 10.0       # Matriz aleatória com entradas entre -10 e 10

    L , U, num_operacoes = crout_dolittle(A)               # Decomposição LU de $A$

    tabela.append([tam, num_operacoes])

    # print(f"A matriz A é dada por\n{A}")
    # print(f"\nA matriz L é dada por\n{L}")
    # print(f"\nA matriz U é dada por\n{U}")
    # print(f"\n\nO produto LU é dado por\n{L@U}")
    # print(f"---------------------------------")

print(f"\nTabela relacionando a Ordem da matriz e o número de operações")
print(f"\n|{'Ordem':^10}|{'Num. Op.':^10}|")

for linha in tabela:    
    print(f"|{linha[0]:^10}|{linha[1]:^10}|")


Tabela relacionando a Ordem da matriz e o número de operações

|  Ordem   | Num. Op. |
|    2     |    4     |
|    3     |    15    |
|    4     |    35    |
|    5     |    66    |
|    6     |   110    |
|    7     |   169    |
|    8     |   245    |
|    9     |   340    |
|    10    |   456    |


> 4. Utilize as funções criadas no passo anterior para criar uma função que recebe uma matriz $A$, um inteiro positivo $n$ e um vetor $b$ e
>     - Decompõe a matriz $A$ como $A=LU$.
>     - Resolve o sistema linear $A^n x = b$, com aplicações sucessivas das funções criadas nos passos 1. e 2.
>     - Conta o número de operações.

In [63]:
def matmul(A,B):
    """Produto de matrizes quadradas de mesma ordem

    Parametros obrigatorios
    ----------
    A , B : Array-like de dimensao 2
        Matrizes quadradas de mesma ordem

    Saída
    ----------
    C : Array-like de dimensão 2
        Produto C = AB
        
    Num_op : Numero de operacoes"""

    num_op = 0    
    n=A.shape[0]
    
    C = np.zeros((n,n))
    
    for i in range(n):
        for j in range(n):
            for k in range(n):
                C[i,j] += A[i,k]*B[k,j]
                num_op += 2
                
    return C, num_op

In [64]:
def decompoe_matriz (A, n, b):
    '''Função decompor matriz em produto LU e calcular o sistema A^n x = b
    
    Parametros obrigatorios
    ----------
    A : Array-like
        Matriz quadrada

    n : int
        Valor do expoente no cálculo

    b : Array-like de dimensão 1
        Vetor independente'''
    num_op_total = 0
    print(f"A:\n", A)
    
    # Primeiro Item
    L, U, num_op = crout_dolittle(A)
    num_op_total += num_op
    print(f"\nLU:\n", L@U)

    # Segundo Item
    A_ = A.copy()
    for i in range(1, n):
        A_, num_op = matmul(A_, A)
        num_op_total += num_op
    
    y, num_op = solve_triangular_inferior(L, b)
    num_op_total += num_op

    x, num_op = solve_triangular_superior(U, y)
    num_op_total += num_op

    print("Ax = b")
    print(f"\nAx : \n", A@x)
    print(f"\nb: \n",b)

In [65]:
ordem = 2
A = 20*np.random.rand(ordem,ordem) -10 # Matriz triangular aleatória com entradas em [-10,10]
b = b = 20*np.random.rand(ordem) - 10 # Vetor aleatório com entradas em [-10,10]
n = 2

decompoe_matriz(A, n, b)

A:
 [[-4.6127836  -4.09201371]
 [-2.24754854  1.43661587]]

LU:
 [[-4.6127836  -4.09201371]
 [-2.24754854  1.43661587]]
Ax = b

Ax : 
 [-4.88864779  3.93130464]

b: 
 [-4.88864779  3.93130464]


> 5. Adapte o algoritmo de resolução de sistemas lineares da aula 9 para que ele conte o número de operações realizadas.

> 6. Adapte o algoritmo abaixo, que realiza o produto de duas matrizes $A$ e $B$, para que ele também conte o número de operações realizadas.

In [66]:
def matmul(A,B):
    """Produto de matrizes quadradas de mesma ordem

    Parametros obrigatorios
    ----------
    A , B : Array-like de dimensao 2
        Matrizes quadradas de mesma ordem

    Saída
    ----------
    C : Array-like de dimensão 2
        Produto C = AB
        
    Num_op : Numero de operacoes"""

    num_op = 0    
    n=A.shape[0]
    
    C = np.zeros((n,n))
    
    for i in range(n):
        for j in range(n):
            for k in range(n):
                C[i,j] += A[i,k]*B[k,j]
                num_op += 2
                
    return C, num_op

In [68]:
ordem = 3
A = 20*np.random.rand(ordem,ordem) -10 # Matriz triangular aleatória com entradas em [-10,10]
B = 20*np.random.rand(ordem,ordem) -10 # Matriz triangular aleatória com entradas em [-10,10]

C, num_operacoes =  matmul(A, B)
print(f"A@B: \n{A@B}")
print(f"\nC: \n{C}")
print(f"\nNumero de Operações: {num_operacoes}")

A@B: 
[[ 42.9010145  -30.68079228  -3.58486075]
 [ 50.5541062   15.37432187 -58.40291476]
 [-13.22571974 -43.67494927  33.71685725]]

C: 
[[ 42.9010145  -30.68079228  -3.58486075]
 [ 50.5541062   15.37432187 -58.40291476]
 [-13.22571974 -43.67494927  33.71685725]]

Numero de Operações: 54


> 7. Crie uma matriz aleatória $A$ de ordem $4\times 4$ e um vetor $b$ de tamanho $4$, e resolva o sistema $A^{20}x = b$ por dois modos:
>     - Utilizando o método do passo 4;
>     - Calculando o produto $A^{20}=A\cdot A\cdots A$ com o algoritmo do passo 6, e resolvendo o sistema $(A^n)x = b$ diretamente com o algoritmo do passo 5.
>
>     Compare os números de operações e os resultados.